In [1]:
import numpy as np
import numpy as np 
import pandas as pd
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem import rdMolDescriptors
from rdkit import DataStructs
from ast import literal_eval
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import svm
from sklearn import metrics
from sklearn.svm import SVC
import pickle
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

https://bitbucket.org/kaistsystemsbiology/deepddi/src/master/data/

In [2]:
df = pd.read_csv('Interaction_information.csv')
df.head()

,Interaction type,Description,Subject,DDI type
0,67,#Drug1 can cause a decrease in the absorption ...,2,DDI type 1
1,18,#Drug1 can cause an increase in the absorption...,1,DDI type 2
2,13,The absorption of #Drug2 can be decreased when...,1,DDI type 3
3,3,The bioavailability of #Drug2 can be decreased...,1,DDI type 4
4,62,The bioavailability of #Drug2 can be increased...,1,DDI type 5


In [3]:
map_label = {}
for i in df.index:
    label = df.at[i, 'Interaction type']
    description = df.at[i, 'Description']
    map_label[int(label)] = description

In [4]:
map_label[1]

'#Drug1 may increase the photosensitizing activities of #Drug2.'

In [11]:
ddi_df = pd.read_csv('KnownDDI.csv')
ddi_df['smiles1'] = None
ddi_df['smiles2'] = None
ddi_df

,Drug1,Drug2,Label,smiles1,smiles2
0,DB04571,DB00460,1,None,None
1,DB00855,DB00460,1,None,None
2,DB09536,DB00460,1,None,None
3,DB01600,DB00460,1,None,None
4,DB09000,DB00460,1,None,None
...,...,...,...,...,...
192279,DB00437,DB00492,86,None,None
192280,DB00437,DB09477,86,None,None
192281,DB00437,DB00790,86,None,None
192282,DB00415,DB00437,86,None,None


In [13]:
all_drugs = pd.read_csv('../../Data/all_drugs2.csv',index_col = False)
all_drugs

,drugbank_id,Name,smiles
0,DB00006,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,DB00014,Goserelin,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
2,DB00027,Gramicidin D,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
3,DB00035,Desmopressin,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
4,DB00050,Cetrorelix,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...
...,...,...,...
10625,DB15456,Vericiguat,COC(=O)NC1=C(N)N=C(N=C1N)C1=NN(CC2=CC=CC=C2F)C...
10626,DB15457,Snubh-nm-333 F-18,CNC1=CC=C(C=C1)C1=NC2=C(S1)C=C([18F])C=C2
10627,DB15458,Norleucine,CCCC[C@H](N)C(O)=O
10628,DB15460,AGG-523,CC(C)(CC1=CC=C(F)C=C1)NC(=O)[C@H](CCC(O)=O)NC(...


In [17]:
smile_label = {}
for i in all_drugs.index:
    drugbank_id = all_drugs.at[i, 'drugbank_id']
    smiles = all_drugs.at[i, 'smiles']
    smile_label[str(drugbank_id)] = smiles

In [19]:
c = 0 
drop_list = []
for i in ddi_df.index:
    try: 
        d1 = ddi_df.at[i, 'Drug1']
        d2 = ddi_df.at[i, 'Drug2']
        ddi_df.at[i, 'smiles1'] = smile_label[str(d1)]
        ddi_df.at[i, 'smiles2'] = smile_label[str(d2)]
        c = c + 1
    except:
        drop_list.append(i)
        pass
print(c)

191541


In [24]:
# ddi_df.drop(index=drop_list, inplace = True) # [deleting null rows]
ddi_df

,Drug1,Drug2,Label,smiles1,smiles2
0,DB04571,DB00460,1,CC1=CC2=CC3=C(OC(=O)C=C3C)C(C)=C2O1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
1,DB00855,DB00460,1,NCC(=O)CCC(O)=O,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
2,DB09536,DB00460,1,O=[Ti]=O,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
3,DB01600,DB00460,1,CC(C(O)=O)C1=CC=C(S1)C(=O)C1=CC=CC=C1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
4,DB09000,DB00460,1,CC(CN(C)C)CN1C2=CC=CC=C2SC2=C1C=C(C=C2)C#N,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
...,...,...,...,...,...
192279,DB00437,DB00492,86,O=C1N=CN=C2NNC=C12,CCC(=O)O[C@@H](O[P@](=O)(CCCCC1=CC=CC=C1)CC(=O...
192280,DB00437,DB09477,86,O=C1N=CN=C2NNC=C12,[H][C@@](C)(N[C@@]([H])(CCC1=CC=CC=C1)C(O)=O)C...
192281,DB00437,DB00790,86,O=C1N=CN=C2NNC=C12,[H][C@]12C[C@H](N(C(=O)[C@H](C)N[C@@H](CCC)C(=...
192282,DB00415,DB00437,86,[H][C@]12SC(C)(C)[C@@H](N1C(=O)[C@H]2NC(=O)[C@...,O=C1N=CN=C2NNC=C12


In [25]:
# ddi_df.to_csv('processed_df.csv')